In [79]:
import pandas as pd
import numpy as np
import dotenv
import ast
import pandera as pa
import incawrapper
from incawrapper import utils
from incawrapper import visualization
import pathlib
import matplotlib.pyplot as plt
import pytest
import seaborn as sns

## TODO
- [x] Get overview of the different files
- [ ] Manually run the provided files and validate that they match the reported results
- [ ] Collect all setting required to replicate
- [ ] Create INCA script

In [190]:
data_file = pathlib.Path('./Literature data/Wirth et al/1-s2.0-S1096717622001197-mmc2.xlsx')
sc3_original_fluxmap_file = pathlib.Path('./Literature data/Wirth et al/INCA files/20220814_SCA3_Fluxmap_After_Revision.mat')
kt2440_original_fluxmap_file = pathlib.Path('./Literature data/Wirth et al/INCA files/P. putida KT2440 Flux Map.mat')
pd.ExcelFile(data_file).sheet_names

['MDVs', 'Physiological parameters', 'Atom transition map and Fluxes']

The original data is heavily formatted in excel, therefore we manually extracted the flux measurements from the data file (docs/examples/Literature data/Wirth et al/1-s2.0-S1096717622001197-mmc2.xlsx, sheet name `Physiological parameters`)

In [93]:
flux_measurement = pd.DataFrame(
    {
        "experiment_id": ["KT2440"]*4,
        "rxn_id" : ["R1", "R4", "R7", "R69"],
        "flux" : [10.45, 0.19, 0.00, 0.654],
        'flux_std_error' : [2.41, 0.07, 0.19, 0.034],
    }
)
incawrapper.dataschemas.FluxMeasurementsSchema.validate(flux_measurement)

,experiment_id,rxn_id,flux,flux_std_error
0,KT2440,R1,10.450,2.410
1,KT2440,R4,0.190,0.070
2,KT2440,R7,0.000,0.190
3,KT2440,R69,0.654,0.034


## Parsing atom map model
The study uses two _P. putida_ strains which has different reactions available. For this example we will only use the data from one of the strains (KT2440).

In [53]:
reactions_KT2440 = (pd.ExcelFile(data_file)
    .parse('Atom transition map and Fluxes', skipfooter=3) # last 3 rows contains summary stats 
    [['Reaction ID', 'Equations (Carbon atom transition)', 'KT2440']]
)
reactions_KT2440.dropna(axis=0, inplace=True)
reactions_KT2440.rename(columns={'Reaction ID': 'rxn_id', 'Equations (Carbon atom transition)':'rxn_eqn'}, inplace=True)
reactions_KT2440.drop(columns='KT2440', inplace=True)
incawrapper.dataschemas.ReactionsSchema.validate(reactions_KT2440)

,rxn_id,rxn_eqn
1,R1,Gluc.ext (abcdef) -> Gluc.per (abcdef)
2,R2,Gluc.per (abcdef) + 2*ATP -> G6P (abcdef)
3,R3,Gluc.per (abcdef) -> Gluco.per (abcdef) + UQH2
4,R4,Gluco.per (abcdef) <-> Gluco.ext (abcdef)
5,R5,Gluco.per (abcdef) + ATP -> 6PG (abcdef)
...,...,...
80,R80,SO4.ext -> SO4
81,R81,O2.ext -> O2
82,R82,OAA (abcd) -> Pyr (abc) + CO2 (d)
83,R83,OAA (abcd) + ATP -> PEP (abc) + CO2 (d)


## Parsing MS data

In [139]:
ms_data = (pd.ExcelFile(data_file)
    .parse('MDVs', header=[0,1,2], index_col=[0,1])
)
# drop spacer rows from excel formatting
ms_data = ms_data[~(ms_data.index == (np.NAN, np.NAN))]
ms_data_long = ms_data.stack(level=[0,1]).reset_index()
ms_data_long.rename(
    columns={
        'level_0':'ms_id', 
        'level_1':'mass_isotope', 
        'level_3' : 'tracers_used',
        'Mass isotopomers': 'strain'
    }, inplace=True
)
ms_data_long['strain'] = ms_data_long['strain'].str.replace('P. putida ', '')

# make compatible with id use in the inca files from authors
ms_data_long['ms_id'] = (ms_data_long['ms_id']
    .str.replace('_', '') 
    .str.lower()
)

In [138]:
ms_data_long

,ms_id,mass_isotope,strain,tracers_used,5 % Absolute uncertainty,Average,Standard deviation
0,ala232,m,KT2440,100% 12C Glucose,0.037880,0.757599,0.037880
1,ala232,m,KT2440,100% 3-13C Glucose,0.015985,0.319700,0.001396
2,ala232,m,KT2440,100% 4-13C Glucose,0.033620,0.672394,0.001383
3,ala232,m,KT2440,50%:50% U-13C:12C Glucose,0.020133,0.402662,0.001941
4,ala232,m,SCA3,100% 12C Glucose,0.038116,0.762313,0.038116
...,...,...,...,...,...,...,...
1338,glucose560,m+6,KT2440,50%:50% U-13C:12C Glucose,0.008821,0.176414,0.011753
1339,glucose560,m+6,SCA3,100% 12C Glucose,0.000407,0.008135,0.001394
1340,glucose560,m+6,SCA3,100% 3-13C Glucose,0.000410,0.008194,0.001925
1341,glucose560,m+6,SCA3,100% 4-13C Glucose,0.000769,0.015387,0.004173


The fragment description is not available from the article data, but we have contacted the authors and obtained their INCA files. Currently, the INCAResults object does not parse the experiment data, but we can still find the information in the `.model.raw` property. We will create an ad-hoc object to parse this data.

In [127]:
class INCAExperiment():
    def __init__(self, exp_dict) -> None:
        # assert exp_dict.keys == ['data_cxn', 'data_flx', 'data_ms', 'data_msms', 'data_nmr', 'id', 'notes', 'on', 'tracers', 'base']

        self.exp_id = exp_dict['id']
        self.n_pool_size_measurements = len(exp_dict['data_cxn'])
        self.n_flux_measurements = len(exp_dict['data_flx'])
        self.n_ms = len(exp_dict['data_ms'])
        self.n_msms = len(exp_dict['data_msms'])
        self.active = exp_dict['on']

        self._raw = exp_dict
    
    @property
    def ms_data(self) -> pd.DataFrame:
        df = pd.DataFrame.from_records(self._raw['data_ms'])
        df['experiment_id'] = self.exp_id
        return df

        

In [145]:
sca3_original_inca_res = incawrapper.INCAResults(inca_matlab_file=sc3_original_fluxmap_file)
ms_dfs_from_inca_files = []
for experiment_dict in sca3_original_inca_res.model.raw['expts']:
    exp = INCAExperiment(experiment_dict)
    df = exp.ms_data
    df['strain'] = 'SCA3'
    ms_dfs_from_inca_files.append(df)

ms_data_from_inca_files = pd.concat(ms_dfs_from_inca_files)
ms_data_from_inca_files.rename(columns={
    'state' : 'met_id',
    'id' : 'ms_id',
    'more' : 'unlabelled_atoms',
    'atoms' : 'labelled_atom_ids'
}, inplace=True)

# process data to match data from article and incawrapper requirements
ms_data_from_inca_files['labelled_atom_ids'] = ms_data_from_inca_files['labelled_atom_ids'].str.split(' ')
ms_data_from_inca_files['unlabelled_atoms'] = ms_data_from_inca_files['unlabelled_atoms'].str.replace(pat = ' ', repl='')
ms_data_from_inca_files['ms_id'] = ms_data_from_inca_files['ms_id'].str.lower()
# cleaning
del ms_dfs_from_inca_files


In [146]:

fragment_info = (
    ms_data_from_inca_files[['labelled_atom_ids', 'ms_id', 'unlabelled_atoms']]
    .drop_duplicates(subset='ms_id')
)
fragment_info

,labelled_atom_ids,ms_id,unlabelled_atoms
0,"[2, 3]",ala232,C8H26ONSi2
1,"[1, 2, 3]",ala260,C8H26O2NSi2
2,"[1, 2]",asp302,C12H32O2NSi2
3,"[2, 3, 4]",asp390,C14H40O3NSi3
4,"[1, 2, 3, 4]",asp418,C14H40O4NSi3
5,"[2, 3, 4, 5]",glu330,C12H36O2NSi2
6,"[1, 2, 3, 4, 5]",glu432,C14H42O4NSi3
7,[2],gly218,C8H24ONSi2
8,"[1, 2]",gly246,C8H24O2NSi2
9,"[2, 3, 4, 5, 6]",ile274,C8H32ONSi2


In [123]:
incawrapper.utils.present_schema_overview(incawrapper.MSMeasurementsSchema)

,column name,dtype,required,nullable,description
0,experiment_id,str,True,False,"ID of the experiment. Must be a valid MATLAB variable name, legal characters are a-z, A-Z, 0-9, and the underscore character."
1,met_id,str,True,False,Metabolite ID of metabolite which is directly measured or from which the fragment is derived through a derivatization method.
2,ms_id,str,True,False,ID of the measured ms fragment - often multiple fragment can be measured from the same metabolite
3,measurement_replicate,int64,True,False,"Replicate number of the measurement of the same fragment in the same experiment. \n""In most cases, the data will only have one measurement per fragment per experiment."
4,labelled_atom_ids,object,True,False,List of atom ids of the labelled atoms in the metabolite.
5,unlabelled_atoms,str,False,True,The molecular formula of the all atoms that cannot be labelled through \nthe introduced labels in the tracers. This typically includes non-carbon elements of the fragment and all elements originating from derivatization agent. \nINCA uses the unlabelled atoms to correct for natural abundance.
6,mass_isotope,int64,True,False,"The mass isotopomer of the fragment.\nE.g. M0, M+1, etc. Specified as an integer. It is allowed to have gaps in the isotopmer of a given fragment, e.g. 0, 2, 3. In this case the intensity and \nstd error of missing isotopomers are filled with NaN before inserted in INCA."
7,intensity,float64,True,True,The measured intensity of the fragment mass isotope.
8,intensity_std_error,float64,True,True,The standard error of the measured intensity of the fragment mass isotope.
9,time,float64,True,False,Time point of measurement only relevant for isotopically non-stationary MFA analysis


We are missing the fragments description. Maybe it could be inferred through tracking through references. However currently Im waiting to see if we get the INCA files.

## Parsing their results

In [56]:
article_flux_dist_raw = (pd.ExcelFile(data_file)
    .parse('Atom transition map and Fluxes', skipfooter=3, header=[0,1], index_col=[0,1]) # last 3 rows contains summary stats 
)
article_flux_dist_raw.index.names = ['rxn_id', 'rxn_eqn']
article_flux_dist_raw.columns.names = ['strain', None]
article_flux_dist = article_flux_dist_raw.stack(level=0).reset_index()

In [49]:
article_flux_dist

,rxn_id,rxn_eqn,strain,Best fit,Lower bound (LB),Standard deviation,Upper bound (UP)
0,R1,Gluc.ext (abcdef) -> Gluc.per (abcdef),KT2440,100.0000,100.0000,0.0000,100.0000
1,R1,Gluc.ext (abcdef) -> Gluc.per (abcdef),SCA3,100.0000,100.0000,0.0000,100.0000
2,R2,Gluc.per (abcdef) + 2*ATP -> G6P (abcdef),KT2440,10.9876,9.1278,0.9934,13.0785
3,R2,Gluc.per (abcdef) + 2*ATP -> G6P (abcdef),SCA3,9.8570,8.0279,0.9831,11.8339
4,R3,Gluc.per (abcdef) -> Gluco.per (abcdef) + UQH2,KT2440,89.0124,86.9215,0.9934,90.8722
...,...,...,...,...,...,...,...
165,R84,F6P (abcdef) -> AcP (ba) + E4P (cdef),SCA3,18.4752,7.8499,8.6501,28.5526
166,R85,X5P (abcde) -> AcP (ba) + GAP (cde),SCA3,11.8567,3.1631,8.5045,21.9152
167,R86,AcP (ba) <-> AcCoA (ba),SCA3,30.3320,26.7082,1.5637,33.9039
168,R87,Pyr (abc) -> Pyr.ext (abc),KT2440,0.0100,0.0100,0.0000,0.0100


In [171]:
sca3_original_inca_res = incawrapper.INCAResults(inca_matlab_file=sc3_original_fluxmap_file)
sca3_original_fluxdist = sca3_original_inca_res.fitdata.fitted_parameters.query("type.str.contains('flux')").copy()
sca3_original_fluxdist['id'].

,type,id,eqn,val,std,lb,ub,unit,free,alf,chi2s,cont,cor,cov,vals,base
0,Net flux,R1,Gluc.ext -> Gluc.per,100.000000,0.000000,[],[],[],0,0.05,[],1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],{'id': []}
1,Net flux,R2,Gluc.per + 2*ATP -> G6P,11.755788,1.050698,[],[],[],0,0.05,[],1,"[0.0, 0.9999999999999998, -0.9999999999999998,...","[0.0, 1.1039666753121549, -1.1039666753121549,...",[],{'id': []}
2,Net flux,R3,Gluc.per -> Gluco.per + UQH2,88.244212,1.050698,[],[],[],0,0.05,[],1,"[0.0, -0.9999999999999998, 0.9999999999999998,...","[0.0, -1.1039666753121549, 1.1039666753121549,...",[],{'id': []}
3,Net flux,R4,Gluco.per -> Gluco.ext,3.700000,0.000000,[],[],[],0,0.05,[],1,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],{'id': []}
4,Net flux,R5,Gluco.per + ATP -> 6PG,25.635057,21703.378704,[],[],[],1,0.05,[],1,"[0.0, -5.3159379712837017e-05, 5.3159379712837...","[0.0, -1.212230579396088, 1.212230579396088, 0...",[],{'id': []}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Net flux,R84,F6P -> AcP + E4P,2.198637,9.308031,[],[],[],1,0.05,[],1,"[0.0, -0.5493745289778411, 0.5493745289778411,...","[0.0, -5.372844907809908, 5.372844907809908, 0...",[],{'id': []}
104,Net flux,R85,X5P -> AcP + GAP,25.192300,8.837047,[],[],[],0,0.05,[],1,"[0.0, 0.5549117855372969, -0.5549117855372969,...","[0.0, 5.152394331099692, -5.152394331099692, 0...",[],{'id': []}
105,Net flux,R86 net,AcP <-> AcCoA,27.390938,1.367093,[],[],[],0,0.05,[],1,"[0.0, -0.15347416867969593, 0.1534741686796959...","[0.0, -0.2204505767102158, 0.2204505767102158,...",[],{'id': []}
106,Exch flux,R86 exch,AcP <-> AcCoA,0.002765,99984.242347,[],[],[],1,0.05,[],1,"[0.0, -1.983516354967813e-06, 1.98351635496781...","[0.0, -0.20837486318478793, 0.2083748631847879...",[],{'id': []}


In [191]:
kt2440_original_inca_res = incawrapper.INCAResults(inca_matlab_file=kt2440_original_fluxmap_file)
kt2440_original_inca_res.fitdata.fitted_parameters.query("type.str.contains('flux')").head()

,type,id,eqn,val,std,lb,ub,unit,free,alf,chi2s,cont,cor,cov,vals,base
0,Net flux,R1,Gluc.ext -> Gluc.per,100.000000,0.0,[],[],[],0,0.05,[],1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],{'id': []}
1,Net flux,R2,Gluc.per -> G6P,31.770819,inf,[],[],[],0,0.05,[],1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, inf, -inf, -inf, inf, 0.0, 0.0, 0.0, 0.0...",[],{'id': []}
2,Net flux,R3,Gluc.per -> Gluco.per,68.229181,inf,[],[],[],0,0.05,[],1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, -inf, inf, inf, -inf, 0.0, 0.0, 0.0, 0.0...",[],{'id': []}
3,Net flux,R4,Gluco.per -> Gluco.ext,55.573039,inf,[],[],[],0,0.05,[],1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, -inf, inf, inf, -inf, -inf, -inf, inf, i...",[],{'id': []}
4,Net flux,R5,Gluco.per -> 6PG,8.445410,inf,[],[],[],0,0.05,[],1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, inf, -inf, -inf, inf, 0.0, inf, -inf, -i...",[],{'id': []}


### Does the provided INCA results match the reported?

In [204]:
class ComparisonOfFluxDist():
    def __init__(self) -> None:
        self.inca_results_tuple = ()
        return None

    def set_article_data(self, df: pd.DataFrame) -> None:
        self.article_results = df
    
    def add_inca_results(self, inca_results_tuple: tuple)-> None:
        self.inca_results_tuple = (*self.inca_results_tuple, inca_results_tuple)
    
    def _merge_data(self, idx_inca_results):
        strain, inca_res = self.inca_results_tuple[idx_inca_results]

        inca_flux_dist = (inca_res
            .fitdata.fitted_parameters
            .query("type.str.contains('Net flux')")
            .assign(rxn_id = lambda x: x['id'].str.replace(' net', ''))
            [['rxn_id', 'val', 'std']]
        )
        df = pd.merge(
            self.article_results.query("strain == @strain"),
            inca_flux_dist,
            on='rxn_id',
            how='right'
        ) 
        return df
    
    def calculate_diff(self, idx_inca_res)-> pd.DataFrame:
        df = self._merge_data(idx_inca_res)
        df = df.assign(
            diff = lambda x: x['Best fit'] - x['val'],
            rel_diff = lambda x: x['diff'] / x['Best fit']
        )
        return df

    def incorrect_fluxes(self, idx_inca_res, quantity, cutoff)->pd.DataFrame: 
        df = self.calculate_diff(idx_inca_res)
        return df[df[quantity] > cutoff]

    def diff_fraction_correct(self, idx_inca_res, cutoff)-> float:
        df = self.calculate_diff(idx_inca_res)
        return (df['diff'] < cutoff).sum() / len(df['diff'])

    def reldiff_fraction_correct(self, idx_inca_res, cutoff)-> float:
        df = self.calculate_diff(idx_inca_res)
        return (df['rel_diff'] < cutoff).sum() / len(df['diff'])


results_compare = ComparisonOfFluxDist()        
results_compare.set_article_data(article_flux_dist)
results_compare.add_inca_results(('SCA3', sca3_original_inca_res))
results_compare.add_inca_results(('KT2440', kt2440_original_inca_res))
results_compare.diff_fraction_correct(0, 0.001)
results_compare.incorrect_fluxes(0, 'diff', 0.001)


,rxn_id,rxn_eqn,strain,Best fit,Lower bound (LB),Standard deviation,Upper bound (UP),val,std,diff,rel_diff
2,R3,Gluc.per (abcdef) -> Gluco.per (abcdef) + UQH2,SCA3,90.1430,8.816610e+01,0.9831,91.9721,8.824421e+01,1.050698e+00,1.898788,0.021064
3,R4,Gluco.per (abcdef) <-> Gluco.ext (abcdef),SCA3,3.7200,3.720000e+00,0.0000,3.7200,3.700000e+00,0.000000e+00,0.020000,0.005376
5,R6,Gluco.per (abcdef) -> Kgluco.per (abcdef) + FADH2,SCA3,72.8527,3.304000e+01,4.5356,87.6618,5.890915e+01,2.170338e+04,13.943546,0.191394
7,R8,Kgluco.per (abcdef) + ATP -> KGluco6P (abcdef),SCA3,39.8127,-2.131600e-14,4.5356,54.6218,2.586915e+01,2.170338e+04,13.943546,0.350229
8,R9,KGluco6P (abcdef) + NADPH -> 6PG (abcdef),SCA3,39.8127,0.000000e+00,4.5356,54.6218,2.586915e+01,2.170338e+04,13.943546,0.350229
10,R11,FBP (abcdef) -> F6P (abcdef),SCA3,18.2170,1.740060e+01,0.4170,19.1000,1.660790e+01,5.062509e-01,1.609096,0.088329
18,R19,G6P (abcdef) -> 6PG (abcdef) + NADPH,SCA3,31.7918,2.868670e+01,1.6123,35.1510,2.876635e+01,1.710057e+00,3.025455,0.095165
19,R20,6PG (abcdef) -> Ri5P (bcdef) + CO2 (a) + NADPH,SCA3,39.9816,3.648510e+01,1.8555,43.9022,3.308710e+01,2.465834e+00,6.894499,0.172442
20,R21,Ri5P (abcde) <-> X5P (abcde),SCA3,34.4754,3.178610e+01,1.3963,37.3611,2.818959e+01,1.645031e+00,6.285809,0.182327
21,R22,Ri5P (abcde) <-> R5P (abcde),SCA3,5.5062,4.157100e+00,0.6600,8.0193,4.897511e+00,1.096597e+00,0.608689,0.110546
